In [ ]:
# Wiring:
#  BNO055 orientation accelerometer to ESP32: Vin=PS1=3V3, GND=GND, SDA=RX2, SCL=TX2
#  PX4AIRSPEEDV1.2i2c = I2C(scl=Pin(15, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP), freq=450000)
#  SHT31D = I2C Humitidy and temperature sensor

In [104]:
%serialconnect


Sent 32 lines (969 bytes) to utils.py.


In [10]:
%sendtofile config.txt

pinled       2
deviceletter @
connection0  JulianS9,beckaaaa,192.168.43.1,9042
dev1         1,rx16,tx17,Z

Sent 4 lines (105 bytes) to config.txt.


In [7]:
%sendtofile --source BNO055_class.py
%sendtofile --source UDPblackviewphone.py
%sendtofile --source utils.py

Sent 42 lines (1123 bytes) to UDPblackviewphone.py.


In [96]:
%sendtofile scani2cdevices.py

from machine import I2C, Pin
import time

px4readrate = 400
sht31readrate = 350

i2c = I2C(scl=Pin(15, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP), freq=450000)
i2cdevices = i2c.scan()
print("i2cdevices", i2cdevices)

# Pitot tube sensor
xbs = None
if 0x28 in i2cdevices:
    xbs = bytearray("Xt00000000d0000r0000\n")
    mxbs = memoryview(xbs)
    tstamppx4 = 0
    print("PX4 pitot found")
else:
    print("PX4 pitot not found")


# Humidity temperature sensor
sbs = None
def crc8(b0, b1):
    crc = 0xFF
    crc ^= b0;
    for i in range(8):
        crc = (((crc << 1) ^ 0x31) if (crc & 0x80) else (crc << 1)) & 0xFF
    crc ^= b1
    for i in range(8):
        crc = (((crc << 1) ^ 0x31) if (crc & 0x80) else (crc << 1)) & 0xFF
    return crc
    
if 0x44 in i2cdevices:
    try:
        i2c.writeto(0x44, b'\xF3\x2D')    # SHT31_READSTATUS
        k = i2c.readfrom(0x44, 3)
        print(k, hex(crc8(k[0], k[1])))
        i2c.writeto(0x44, b'\x30\xA2')    # SHT31_SOFTRESET
        time.sleep(0.1)
        i2c.writeto(0x44, b'\x27\x37')    # read 10Hz?
        time.sleep(0.1)
        i2c.writeto(0x44, b'\xF3\x2D')    # SHT31_READSTATUS
        k = i2c.readfrom(0x44, 3)
        print(k, hex(crc8(k[0], k[1])))
    except OSError:
        pass
    print("SHT31D found")
    sbs = bytearray("St00000000r0000a0000\n")
    msbs = memoryview(sbs)
    tstampsht31 = 0
else:
    print("SHT31D not found")

    
def readi2cdevices(tstamp):
    global tstamppx4, tstampsht31
    if xbs != None:
        if tstamppx4 == 0:
            tstamppx4 = tstamp + px4readrate
            x = i2c.readfrom(0x28, 4)
            #status = x[0] & 0xC0  # should be zero
            rawpress = ((x[0] & 0xFF) << 8) | (x[1])
            #rawpress = ((x[0] & 0x3F) << 8) | (x[1])
            rawtemp = (x[2] << 3) | (x[3] >> 5)
            mxbs[2:10] = b"%08X" % tstamp
            mxbs[11:15] = b"%04X" % rawpress
            mxbs[16:20] = b"%04X" % rawtemp
            return xbs
            #psi = rawpress*(1.0/(0x3FFF*0.4)) - 1.25
            #temp = rawtemp*200.0/0x7FF - 50
        elif tstamp > tstamppx4:
            tstamppx4 = 0
            i2c.writeto(0x28, b"")

    if sbs != None:
        if tstamp > tstampsht31:
            tstampsht31 = tstamp + sht31readrate
            i2c.writeto(0x44, b'\xE0\x00')
            k = i2c.readfrom(0x44, 6)
            if k[2] == crc8(k[0], k[1]) and k[5] == crc8(k[3], k[4]):
                rawtemp = (k[0]<<8) | k[1]
                rawhumid = (k[3]<<8) | k[4]
                msbs[2:10] = b"%08X" % tstamp
                msbs[11:15] = b"%04X" % rawhumid
                msbs[16:20] = b"%04X" % rawtemp

                stemp = (rawtemp * 175)/0xFFFF - 45
                shumid = (rawhumid*100)/0xFFFF
                print(stemp, shumid)
                return sbs
            
    return None




Sent 95 lines (2848 bytes) to scani2cdevices.py.


In [97]:
#while True:
#    tstamp = time.ticks_ms()
#    v = readi2cdevices(tstamp)
#    if v is not None:
#        print(v)


In [99]:
%sendtofile main.py

from machine import UART, Pin
import time, socket, urandom
from BNO055_class import BNO055
from UDPblackviewphone import connectActivePhone
from scani2cdevices import readi2cdevices

fconfig = dict(x.strip().split(None, 1)  for x in open("config.txt"))
pinled = Pin(int(fconfig["pinled"]), Pin.OUT)

def MakeBNO055(dev):
    d = dev.split(",")
    u = UART(int(d[0]), baudrate=115200, rx=int(d[1][2:]), tx=int(d[2][2:]))
    for i in range(5):
        try:
            print(u, d[3])
            return BNO055(u, d[3])
        except OSError as e:
            print(e)
        print("fail", i, d)
        time.sleep(0.5)
dev1 = MakeBNO055(fconfig["dev1"])


deviceletter = fconfig["deviceletter"]
c = fconfig["connection0"].split(",")
hotspots = { c[0].encode(): (c[1].encode(), c[2], int(c[3])) }
androidipnumber = c[2]
portnumber = int(c[3])
print("deviceletter", deviceletter)

while not connectActivePhone(pinled, hotspots):
    for j in range(3):
        for i in range(7):
            pinled.value(i%2)
            time.sleep_ms(100)
        time.sleep_ms(800)

import socket, urandom
prevflushstamp = 0
nextledonstamp = 0

ubs = bytearray("Ut00000000i00\n")
mubs = memoryview(ubs)

while True:
    try:
        ss = socket.socket()
        ss.settimeout(1)
        print(ss)
        ss.connect(socket.getaddrinfo(androidipnumber, portnumber)[0][-1])
        s = ss.makefile('rwb', 0)
        print(s.readline())
        s.write(b"%c%c%c%c"%(deviceletter,deviceletter,deviceletter,deviceletter))
        dwrite = s.write
        while True:
            tstamp = time.ticks_ms()
            if tstamp > nextledonstamp:
                pv = (0 if pinled.value() else 1)
                pinled.value(pv)
                mubs[2:10] = b"%08X" % tstamp
                mubs[12] = 48+pv #'0'or'1'
                dwrite(ubs)
                nextledonstamp = tstamp + urandom.randint(5000, 10000)//(10 if pv else 1)
            if dev1 is not None:
                b1 = dev1.readhexlifyBNO055(10)
                if b1 is not None:
                    dwrite(b1)

            try:
                bs = readi2cdevices(tstamp)
                if bs is not None:
                    dwrite(bs)
            except OSError:
                pass
                
    except OSError as e:
        print("OSError", e)
        time.sleep(2)


Sent 78 lines (2330 bytes) to main.py.
